In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
import gensim.downloader as api

# Load pre-trained FastText embeddings from Gensim
fasttext_model = api.load("fasttext-wiki-news-subwords-300")

embedding_dim = 300  # FastText embeddings have 300 dimensions

[==================================================] 100.0% 958.5/958.4MB downloaded


In [ ]:
# Load IMDb dataset
vocab_size = 5000  # Limiting the vocabulary size to 20k most frequent words
max_length = 128    # Max length of input sequences

# Load the dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=vocab_size)

# Pad sequences to ensure uniform input length
X_train = pad_sequences(X_train, maxlen=max_length)
X_test = pad_sequences(X_test, maxlen=max_length)

# Create an embedding matrix for words in IMDb's vocabulary using FastText embeddings
embedding_matrix = np.zeros((vocab_size, embedding_dim))

word_index = tf.keras.datasets.imdb.get_word_index()

for word, index in word_index.items():
    if index < vocab_size:
        embedding_vector = None
        try:
            embedding_vector = fasttext_model.get_vector(word)
        except KeyError:
            pass  # Skip words not found in the FastText embeddings
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [35]:
# Create the embedding layer with the pre-trained FastText vectors
embedding_layer = tf.keras.layers.Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], trainable=False)  # Set trainable=False to freeze the embeddings

 # Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=128),  # Input layer (embedding)
    tf.keras.layers.Conv1D(filters=128, kernel_size=5, activation='relu', kernel_regularizer=l2(0.01)),        # 1D Convolutional layer
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.GlobalMaxPooling1D(),                                         # Global Max Pooling
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=l2(0.05)),   # Fully connected layer
    tf.keras.layers.Dropout(0.5),                                                 # Dropout for regularization
    tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=l2(0.05)),   # Fully connected layer
    tf.keras.layers.Dropout(0.5),                                                 # Dropout for regularization
    tf.keras.layers.Dense(1, activation='sigmoid')                                # Output layer (binary classification)
])

# Compile the model
model.compile(optimizer='adamax', loss='binary_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [36]:
from sklearn.metrics import f1_score

# Model training
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc}')

y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)
f1 = f1_score(y_test, y_pred_classes)
print(f'F1 Score: {f1}')

Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 14s 10ms/step - accuracy: 0.5085 - loss: 5.7567 - val_accuracy: 0.5161 - val_loss: 2.3057
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - accuracy: 0.5624 - loss: 1.8217 - val_accuracy: 0.7460 - val_loss: 0.8947
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7601 - loss: 0.7450 - val_accuracy: 0.8362 - val_loss: 0.5763
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.8598 - loss: 0.4894 - val_accuracy: 0.8610 - val_loss: 0.5014
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.8883 - loss: 0.4275 - val_accuracy: 0.8584 - val_loss: 0.4809
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9156 - loss: 0.3667 - val_accuracy: 0.8309 - val_loss: 0.5031
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.9301 - loss: 0.3333 - val_accuracy: 0.8592 - val_loss: 0.4527
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9473 - loss: 0.2927 - val_accuracy: